# 前処理、形態素解析、ベクトル化

処理の流れ

生文 → 形態素解析 → 数値化（ベクトル化）

独立請求項の生文が入力されている列の1行目に「独立請求項」という名前がつけられているexcelデータを 

'sample_data.xlsx' として 'data/'フォルダに保存しておけばサンプルプログラムが動作します。

In [ ]:
import os
from google.colab import drive

# googleドライブのマウント 
drive.mount('/content/drive/')

# 作業ファイルをマイドライブに変更
os.chdir('/content/drive/My Drive/') 

Mounted at /content/drive/


In [ ]:
# pandasモジュールを用いたサンプルデータの読み込み

import pandas as pd

data = pd.read_excel('data/original_data.xlsm', sheet_name= 'text')
data.head()

,公報番号(全文リンク),発明等の名称,出願人・権利者(最新),ＩＰＣ(最新),ＦＩ(最新),Ｆターム(最新),請求の範囲(独立請求項),"ＩＰＣ(最新,筆頭)",FI(最新),FI(整理),ラベル,正誤,"FI(整理,重複削除)",錯誤,データ数,錯誤率
0,特開2019-220570,パターン形成方法、パターン形成物およびパターン形成装置,コニカミノルタ株式会社,H05K3/10;C09D11/30;B41M5/00;B41J2/01;B05D1/36;...,H05K 3/10 D;B41J 2/01 501;B41J 2/...,2C056 FD10;2C056 FD01;2C056 FB05;2C056 EC72;2C...,【請求項１】 第１の液体を基材表面に付与されてなる第１液層を形成する工程と、 前記第１液...,H05K3/10,H05K3/10D,H05K3/10,3,0,B41M5/001,83.0,241.0,0.344398
1,特開2019-219949,導電性細線パターンの製造方法及びタッチパネルセンサーの製造方法,コニカミノルタ株式会社,G06F3/041;G06F3/044;H05K3/10;H05K3/24,G06F 3/041 660;G06F 3/044 127;G06F 3/04...,5E343 AA12;5E343 GG20;5E343 AA18;5E343 BB23;5E...,【請求項１】 ２種以上の金属によって構成される導電性細線によって構成される、１０００ｃｍ２...,G06F3/041,G06F3/041660,G06F3/04,3,1,B41M5/00A,69.0,166.0,0.415663
2,特開2019-218636,染色物の製造方法、染色調整インクジェットインク、及び染色調整インクジェットインクセット,株式会社ミマキエンジニアリング,D06P5/00;C09D11/54;C09D11/32;C09D11/40;D06P5/3...,D06P 5/00 120 Z;B41J 2/01 501;B41M 5/...,2C056 FC01;2C056 HA42;2C056 EA04;2C056 FB03;2H...,【請求項１】 染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な...,D06P5/00,D06P5/00120Z,D06P5/00,3,1,C09D11/3,60.0,1122.0,0.053476
3,特開2019-218522,インクジェットインク,ゼネラル株式会社,C09D11/30;C09D11/38;B41J2/01;B41M5/00;A61K9/20...,C09D 11/30;B41J 2/01 501;B41M 5/00 120...,2C056 FC01;2H186 BA08;2H186 DA12;2H186 FB11;2H...,【請求項１】 着色剤、アラビアガム、炭素数１２以上、２０以下の高級アルコール、炭素数１以上...,C09D11/30,C09D11/30,C09D11/3,0,1,B41J2/01,58.0,1109.0,0.052299
4,特開2019-218514,インクジェット記録液セット、インクジェット記録用前処理液の製造方法、印刷物及びインクジェット...,コニカミノルタ株式会社,C09D11/54;C09D11/322;B41J2/01;B41M5/00,C09D 11/54;B41J 2/01 123;B41J 2/01 501...,2C056 FC01;2C056 FB02;2C056 EA13;2C056 EA04;2C...,【請求項１】 少なくとも前処理液とインクとからなるインクジェット記録液セットであって、 ...,C09D11/54,C09D11/54,C09D11/5,0,1,D06P5/00,25.0,44.0,0.568182


# 前処理

[正規表現](https://docs.python.org/ja/3/howto/regex.html)を使い、特許文書に特有の文字列を削除します。

処理した文字列をリスト（processed_data）に収納します。

processed_dataをnumpy形式に変換し、保存します。

また、先頭から10文書を表示します。

In [ ]:
import re
import numpy as np
import pandas as pd

#  正規表現のコンパイル
sep = re.compile('【.*?】|\n|\u3000| |\n') 

processed_data = []
for text in data['請求の範囲(独立請求項)']:
  processed_data.append(sep.sub('', text))
processed_data = np.array(processed_data)
np.save('data/processed_data.npy', processed_data)
processed_data[:10]

array(['第１の液体を基材表面に付与されてなる第１液層を形成する工程と、前記第１液層が形成された前記基材表面に、前記第１の液体とは非相溶であり、かつ、前記第１の液体よりも比重が大きい第２の液体の液滴をインクジェット法で付与する工程と、前記第１の液体および第２の液体の一方の液体を選択的に硬化させる工程と、を有する、パターン形成方法。インクジェット法により基材表面に付与された液体が硬化して形成されたパターン形成物であって、前記液体の硬化物により形成されるパターンが有する前記基材表面に接する壁面と、前記基材表面と、の間の角度は、４５°以上である、パターン形成物。第１の液体を基材表面に付与する第１液体付与部と、前記第１の液体が付与された前記基材表面に、前記第１の液体とは非相溶であり、かつ、前記第１の液体よりも比重が大きい第２の液体の液滴をインクジェット法で付与する第２液体付与部と、前記第１の液体および第２の液体の一方の液体を選択的に硬化させる選択的硬化部と、を有する、パターン形成装置。',
       '２種以上の金属によって構成される導電性細線によって構成される、１０００ｃｍ２以上の面積にわたる導電性細線パターンの製造方法であって、基材上に第１の金属によって構成された第１の導電性細線をパターニングし、次いで、前記第１の導電性細線に、前記第１の金属とは異なる金属種である第２の金属によって構成された導電性皮膜を被覆する被覆処理を施して、前記第１の導電性細線と前記導電性皮膜とによって構成された第２の導電性細線を形成し、次いで、前記第２の導電性細線に、該第２の導電性細線の線幅の減少速度が０．５μｍ／ｍｉｎ以下となるエッチングを施して、前記第２の導電性細線を構成する前記導電性皮膜の一部が除去された第３の導電性細線を形成することを特徴とする導電性細線パターンの製造方法。',
       '染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、色材とを含有する染色調整インクを前記メディアにインクジェット方式により塗布する塗布工程と、前記染色調整インクが塗布された前記メディアを前記染色液に浸漬して染色する染色工程と、を備える、染色物の製造方法。染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、前記染色液

# 形態素解析

文字列を分解し、スペースで区切った新たな文字列を生成します。

## MeCab

日本語の文法に依存した形態素解析器として最も有名な方法です。

MeCabの導入は環境に依存します。

[mecab-python3](https://pypi.org/project/mecab-python3/) （[Python3からMeCabを使う](https://qiita.com/taroc/items/b9afd914432da08dafc8)）

[natto-py](https://pypi.org/project/natto-py/)（[Python の MeCab バインディング natto-py を使う](https://qiita.com/buruzaemon/items/975027cea6371b2c5ec3)）

等を検索して導入してください。

In [ ]:
# MeCabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

numpy形式のテキストデータを読み込み、

MeCabで分かち書きしたものをリスト（result）に収納します。

In [ ]:
import MeCab
import numpy as np
import pandas as pd


m = MeCab.Tagger('-Owakati')

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(m.parse(text))
result = np.array(result)
np.save('data/mecab_data.npy', result)
result[:10]

array(['第 １ の 液体 を 基材 表面 に 付与 さ れ て なる 第 １ 液 層 を 形成 する 工程 と 、 前記 第 １ 液 層 が 形成 さ れ た 前記 基材 表面 に 、 前記 第 １ の 液体 と は 非 相 溶 で あり 、 かつ 、 前記 第 １ の 液体 より も 比重 が 大きい 第 ２ の 液体 の 液滴 を インク ジェット 法 で 付与 する 工程 と 、 前記 第 １ の 液体 および 第 ２ の 液体 の 一方 の 液体 を 選択 的 に 硬化 さ せる 工程 と 、 を 有する 、 パターン 形成 方法 。 インク ジェット 法 に より 基材 表面 に 付与 さ れ た 液体 が 硬化 し て 形成 さ れ た パターン 形成 物 で あっ て 、 前記 液体 の 硬化 物 に より 形成 さ れる パターン が 有する 前記 基材 表面 に 接する 壁面 と 、 前記 基材 表面 と 、 の 間 の 角度 は 、 ４５ ° 以上 で ある 、 パターン 形成 物 。 第 １ の 液体 を 基材 表面 に 付与 する 第 １ 液体 付与 部 と 、 前記 第 １ の 液体 が 付与 さ れ た 前記 基材 表面 に 、 前記 第 １ の 液体 と は 非 相 溶 で あり 、 かつ 、 前記 第 １ の 液体 より も 比重 が 大きい 第 ２ の 液体 の 液滴 を インク ジェット 法 で 付与 する 第 ２ 液体 付与 部 と 、 前記 第 １ の 液体 および 第 ２ の 液体 の 一方 の 液体 を 選択 的 に 硬化 さ せる 選択 的 硬化 部 と 、 を 有する 、 パターン 形成 装置 。 \n',
       '２ 種 以上 の 金属 に よっ て 構成 さ れる 導電 性 細線 に よっ て 構成 さ れる 、 １０００ ｃｍ２ 以上 の 面積 に わたる 導電 性 細線 パターン の 製造 方法 で あっ て 、 基材 上 に 第 １ の 金属 に よっ て 構成 さ れ た 第 １ の 導電 性 細線 を パターニング し 、 次いで 、 前記 第 １ の 導電 性 細線 に 、 前記 第 １ の 金属 と は 異なる 金属 種 で ある 第 ２ の 金属 に よっ て 構成 さ れ た 導電

## [sentencepiece](https://pypi.org/project/sentencepiece/)

サブワード法として最も有名な方法です。

あらかじめ学習済みモデルとして、

[BERT with SentencePiece を日本語 Wikipedia で学習してモデルを公開しました｜原理的には可能](https://yoheikikuta.github.io/bert-japanese/)

が公開している [google Drive](https://drive.google.com/drive/folders/1Zsm9DD40lrUVu6iAnIuTH2ODIkh-WM-O) 上のデータ から 'wiki-ja.model' をダウンロードして　'./data'　に保存しておきます。


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.8MB/s 


In [ ]:
import sentencepiece as spm
import numpy as np

sp = spm.SentencePieceProcessor()
sp.load('data/wiki-ja.model')

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(' '.join(sp.EncodeAsPieces(text)))
result = np.array(result)
np.save('data/sentencepiece_data.npy', result)
result[:10]

array(['▁第 1 の 液体 を 基 材 表面に 付与 されて なる 第 1 液 層 を形成する 工程 と 、 前 記 第 1 液 層 が 形成された 前 記 基 材 表面に 、 前 記 第 1 の 液体 とは 非 相 溶 であり 、 かつ 、 前 記 第 1 の 液体 よりも 比重 が大きい 第 2 の 液体 の 液 滴 を インク ジェット 法 で 付与 する 工程 と 、 前 記 第 1 の 液体 および 第 2 の 液体 の 一方 の 液体 を 選択 的に 硬化 させる 工程 と 、 を有する 、 パターン 形成 方法 。 インク ジェット 法により 基 材 表面に 付与 された 液体 が 硬化 して 形成された パターン 形成 物 であって 、 前 記 液体 の 硬化 物 により 形成される パターン が 有 する 前 記 基 材 表面 に接する 壁面 と 、 前 記 基 材 表面 と 、 の間の 角度 は 、 45 ° 以上 である 、 パターン 形成 物 。 第 1 の 液体 を 基 材 表面に 付与 する 第 1 液体 付与 部 と 、 前 記 第 1 の 液体 が付与され た 前 記 基 材 表面に 、 前 記 第 1 の 液体 とは 非 相 溶 であり 、 かつ 、 前 記 第 1 の 液体 よりも 比重 が大きい 第 2 の 液体 の 液 滴 を インク ジェット 法 で 付与 する 第 2 液体 付与 部 と 、 前 記 第 1 の 液体 および 第 2 の 液体 の 一方 の 液体 を 選択 的に 硬化 させる 選択 的 硬化 部 と 、 を有する 、 パターン 形成 装置 。',
       '▁2 種 以上の 金属 によって 構成 される 導 電 性 細 線 によって 構成 される 、 1000 cm 2 以上の 面積 にわたる 導 電 性 細 線 パターン の製造 方法 であって 、 基 材 上に 第 1 の 金属 によって 構成 された 第 1 の 導 電 性 細 線を パ ター ニング し 、 次いで 、 前 記 第 1 の 導 電 性 細 線 に 、 前 記 第 1 の 金属 とは異なる 金属 種である 第 2 の 金属 によって 構成 された 導 電 性 皮 膜 を 被覆 する 被覆 処理 を施し て 、 前 記 第 

In [ ]:
# おまけ　テキストを単語ID列に変換することもできます。
import sentencepiece as spm
import numpy as pd

sp = spm.SentencePieceProcessor()
sp.load('data/wiki-ja.model')
result = []
maxlen = 0
for text in processed_data:
  processed_text = sp.EncodeAsIds(text)
  result.append(processed_text)
  if len(processed_text) > maxlen:
    maxlen = len(processed_text)
# zero padding で文書の長さをを揃える
for i, processed_text in enumerate(result):
  result[i] = result[i]+[0]*(maxlen-len(result[i]))
result = np.array(result)
print(result[:10], '\n', result.shape, '次元')

[[    9 25062  3591 ...     0     0     0]
 [    9   447   502 ...     0     0     0]
 [    9  4694   217 ...     0     0     0]
 ...
 [    9  4694 17230 ...     0     0     0]
 [    9 17230  4965 ...     0     0     0]
 [    9   659 12324 ...     0     0     0]] 
 (50, 304) 次元


## n-gram

単純に n文字 で分割しますが、nが2以上の時、重なり合うように切り出す工夫が必要です。

In [ ]:
n = 3 

def ngram(words, n):
  # n文字ずつ切り出し、1文字×nのタプル生成
  # ('染', '色', '液'), ('色', '液', 'に'), ('液', 'に', 'よ')
  ngram = list(zip(*(words[i:] for i in range(n))))
  # n文字の文字列リストに変換　'染色液', '色液に', '液によ', 'による'
  # リストの要素をスペースで連結して一つの文字列にする
  return ' '.join([''.join(j) for j in ngram])

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(ngram(text, n))
result = np.array(result)
np.save('data/'+str(n)+'gram_data.npy', result)
result[:10]


array(['第１の １の液 の液体 液体を 体を基 を基材 基材表 材表面 表面に 面に付 に付与 付与さ 与され されて れてな てなる なる第 る第１ 第１液 １液層 液層を 層を形 を形成 形成す 成する する工 る工程 工程と 程と、 と、前 、前記 前記第 記第１ 第１液 １液層 液層が 層が形 が形成 形成さ 成され された れた前 た前記 前記基 記基材 基材表 材表面 表面に 面に、 に、前 、前記 前記第 記第１ 第１の １の液 の液体 液体と 体とは とは非 は非相 非相溶 相溶で 溶であ であり あり、 り、か 、かつ かつ、 つ、前 、前記 前記第 記第１ 第１の １の液 の液体 液体よ 体より よりも りも比 も比重 比重が 重が大 が大き 大きい きい第 い第２ 第２の ２の液 の液体 液体の 体の液 の液滴 液滴を 滴をイ をイン インク ンクジ クジェ ジェッ ェット ット法 ト法で 法で付 で付与 付与す 与する する工 る工程 工程と 程と、 と、前 、前記 前記第 記第１ 第１の １の液 の液体 液体お 体およ および よび第 び第２ 第２の ２の液 の液体 液体の 体の一 の一方 一方の 方の液 の液体 液体を 体を選 を選択 選択的 択的に 的に硬 に硬化 硬化さ 化させ させる せる工 る工程 工程と 程と、 と、を 、を有 を有す 有する する、 る、パ 、パタ パター ターン ーン形 ン形成 形成方 成方法 方法。 法。イ 。イン インク ンクジ クジェ ジェッ ェット ット法 ト法に 法によ により より基 り基材 基材表 材表面 表面に 面に付 に付与 付与さ 与され された れた液 た液体 液体が 体が硬 が硬化 硬化し 化して して形 て形成 形成さ 成され された れたパ たパタ パター ターン ーン形 ン形成 形成物 成物で 物であ であっ あって って、 て、前 、前記 前記液 記液体 液体の 体の硬 の硬化 硬化物 化物に 物によ により より形 り形成 形成さ 成され される れるパ るパタ パター ターン ーンが ンが有 が有す 有する する前 る前記 前記基 記基材 基材表 材表面 表面に 面に接 に接す 接する する壁 る壁面 壁面と 面と、 と、前 、前記 前記基 記基材 基材表 材表面 

In [ ]:
n = 2
import numpy as np
def ngram(words, n):
  # n文字ずつ切り出し、1文字×nのタプル生成
  # ('染', '色'), ('色', '液'), ('液', 'に')
  ngram = list(zip(*(words[i:] for i in range(n))))
  # n文字の文字列リストに変換　'染色', '色液', '液に', 'によ'
  # リストの要素をスペースで連結して一つの文字列にする
  return ' '.join([''.join(j) for j in ngram])

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(ngram(text, n))
result = np.array(result)
np.save('data/'+str(n)+'gram_data.npy', result)
result[:10]


array(['第１ １の の液 液体 体を を基 基材 材表 表面 面に に付 付与 与さ され れて てな なる る第 第１ １液 液層 層を を形 形成 成す する る工 工程 程と と、 、前 前記 記第 第１ １液 液層 層が が形 形成 成さ され れた た前 前記 記基 基材 材表 表面 面に に、 、前 前記 記第 第１ １の の液 液体 体と とは は非 非相 相溶 溶で であ あり り、 、か かつ つ、 、前 前記 記第 第１ １の の液 液体 体よ より りも も比 比重 重が が大 大き きい い第 第２ ２の の液 液体 体の の液 液滴 滴を をイ イン ンク クジ ジェ ェッ ット ト法 法で で付 付与 与す する る工 工程 程と と、 、前 前記 記第 第１ １の の液 液体 体お およ よび び第 第２ ２の の液 液体 体の の一 一方 方の の液 液体 体を を選 選択 択的 的に に硬 硬化 化さ させ せる る工 工程 程と と、 、を を有 有す する る、 、パ パタ ター ーン ン形 形成 成方 方法 法。 。イ イン ンク クジ ジェ ェッ ット ト法 法に によ より り基 基材 材表 表面 面に に付 付与 与さ され れた た液 液体 体が が硬 硬化 化し して て形 形成 成さ され れた たパ パタ ター ーン ン形 形成 成物 物で であ あっ って て、 、前 前記 記液 液体 体の の硬 硬化 化物 物に によ より り形 形成 成さ され れる るパ パタ ター ーン ンが が有 有す する る前 前記 記基 基材 材表 表面 面に に接 接す する る壁 壁面 面と と、 、前 前記 記基 基材 材表 表面 面と と、 、の の間 間の の角 角度 度は は、 、４ ４５ ５° °以 以上 上で であ ある る、 、パ パタ ター ーン ン形 形成 成物 物。 。第 第１ １の の液 液体 体を を基 基材 材表 表面 面に に付 付与 与す する る第 第１ １液 液体 体付 付与 与部 部と と、 、前 前記 記第 第１ １の の液 液体 体が が付 付与 与さ され れた た前 前記 記基 基材 材表 表面 面に に、 、前 前記 記第 第１ １の の液 液体 体と とは は非 非相

# ベクトル化

形態素に分割されたテキストを数値化します。

参考：[機械学習 〜 テキスト特徴量（CountVectorizer, TfidfVectorizer） 〜](https://qiita.com/fujin/items/b1a7152c2ec2b4963160)

## Bag of Words

scikit-leran モジュールの [Countvectrizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) を使用して、単語の出現頻度を求めます。

参考：[【python】sklearnのCountVectorizerの使い方｜静かなる名辞](https://www.haya-programming.com/entry/2018/02/25/044525)　ほか多数

注意：

１）すべての文書を読み込んで一気にベクトル化する必要があります。

２）データ量が多いので [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) 形式で出力されます。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['2-gram', 'data/2gram_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word')

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  count_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  savefile = file.replace('_data.npy', '_BoW_csr')
  mmwrite(savefile, csr_matrix(count_vec))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  # 形態素(0-5番目、100-105番目)表示
  print(vectorizer.get_feature_names()[:5], 
        vectorizer.get_feature_names()[100:105], '\n')

MeCabで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (6752, 7505) 次元 最大値 38
['000', '015', '10', '100', '101'] ['my', 'na', 'nf', 'nh', 'no'] 

sentencepieceで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (6752, 6031) 次元 最大値 38
['00', '000', '0000', '01', '02'] ['50', '500', '5000', '51', '52'] 

2-gramで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (6752, 38862) 次元 最大値 53
['00', '01', '02', '03', '05'] ['32', '33', '34', '35', '38'] 

3-gramで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (6752, 148887) 次元 最大値 38
['00', '000', '00k', '00の', '01'] ['1で表', '1と', '1との', '1と同', '1ない'] 



## TF-IDF
scikit-leran モジュールの [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer) を使用して、単語のレア度を求めます。

他の注意事項はCountVectrizer と同様です。

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['2-gram', 'data/2gram_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word')

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  tfidf_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  savefile = file.replace('_data.npy', '_TfIdf_csr')
  mmwrite(savefile, csr_matrix(tfidf_vec))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  # 形態素(0-5番目、100-105番目)表示
  print(vectorizer.get_feature_names()[:5], 
        vectorizer.get_feature_names()[100:105], '\n')

MeCabで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 7505) 次元 最大値 0.9344405466862613
['000', '015', '10', '100', '101'] ['my', 'na', 'nf', 'nh', 'no'] 

sentencepieceで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 6031) 次元 最大値 0.9502702649148659
['00', '000', '0000', '01', '02'] ['50', '500', '5000', '51', '52'] 

2-gramで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 38862) 次元 最大値 0.6923048299717444
['00', '01', '02', '03', '05'] ['32', '33', '34', '35', '38'] 

3-gramで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 

CountVectrizer も TfidfVectorizer も入力文の形態素を自動抽出して計算しますが、

vocabulary に単語のリストを与えると、与えた単語に対して計算します。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vocab = ['備える', 'ノニオン', 'ペロブスカイト', 'ポリエステル', 
         'ポリオレフィン', 'ポリカーボネート', '印刷', '原子', '重なり']

for method, file in files:
  print(method+'で分割した文書をベクトル化', '\nBoW')
  corpus = np.load(file, allow_pickle=True)
  vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', vocabulary=vocab)
  print(vectorizer.get_feature_names())
  count_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  print(count_vec.toarray()[:10], '\n', count_vec.shape, '次元', '最大値', np.max(count_vec.toarray()), '\nTfIdf')
  vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', vocabulary=vocab)
  tfidf_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  print(tfidf_vec.toarray()[:10], '\n', tfidf_vec.shape, '次元', '最大値', np.max(tfidf_vec.toarray()), '\n')


MeCabで分割した文書をベクトル化 
BoW
['備える', 'ノニオン', 'ペロブスカイト', 'ポリエステル', 'ポリオレフィン', 'ポリカーボネート', '印刷', '原子', '重なり']
[[ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  2  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  0  0  0]] 
 (50, 9) 次元 最大値 11 
TfIdf
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.67902226 0.73411768 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         1.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.   

## HashingVectrizer

[feature hashing](https://ja.wikipedia.org/wiki/Feature_Hashing)という手法を使った埋め込み表現です。

scikit-leran モジュールの [HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) を使用して計算します。

単語情報は設定した(n_features)次元に分散されます。ベクトルの要素と単語とは1対1対応ではありません。

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['2-gram', 'data/2gram_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
n_features = [65536]
              #1048576]
              #256]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  for i in range(len(n_features)):
    vectorizer = HashingVectorizer(ngram_range=(1, 1), analyzer='word', n_features=n_features[i])
    hash_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
    savefile = file.replace('_data.npy', '_Hash'+str(n_features[i])+'_csr')
    mmwrite(savefile, csr_matrix(hash_vec))
    # 再読込、表示
    loaddata = mmread(savefile+'.mtx').todense()
    print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  print()

MeCabで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 65536) 次元 最大値 0.8986914687663685

sentencepieceで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 65536) 次元 最大値 0.9128709291752769

3-gramで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 65536) 次元 最大値 0.6801159602108064

3-gramで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (6752, 65536) 次元 最大値 0.7343236168762819



## Doc2Vec

gensim モジュールの [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html) を用いて、

文章中の隣り合う単語の出現頻度を予測する学習を行い、

得られたモデルに未知の文書を入力すると、学習時に設定した次元のベクトルが得られます。

学習に用いたタグを入力すると、学習に用いた文書のベクトルが得られます。

(参考)

[Doc2Vecについてまとめる](https://qiita.com/g-k/items/5ea94c13281f675302ca)

[Doc2Vecの仕組みとgensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html)

[Word2Vecとは | 分散表現・Skip-gram法とCBOWの仕組み・ツールや活用事例まで徹底解説](https://ledge.ai/word2vec/)

In [ ]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
import joblib

files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['2-gram', 'data/2gram_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  training_docs = []
  for i in range(len(corpus)):
    training_docs.append(TaggedDocument(words=corpus[i],
                       tags=[data['公報番号(全文リンク)'][i]]))
  # dm=0 でDoc2Vec 学習
  model = Doc2Vec(documents=training_docs, vector_size=300, min_count=1, dm=0)
  joblib.dump(model, 'data/'+method+'_doc2vec_dm0.model', compress=3)

  # 学習モデルに文書を入力しベクトルを得る
  doc_vec = []
  for i in range(len(data)):
    doc_vec.append(model.infer_vector(training_docs[i].tags))
  savefile = 'data/'+method+'_doc2vec_dm0_csr'
  mmwrite(savefile, csr_matrix(np.array(doc_vec)))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata), '\n')

MeCabで分割した文書をベクトル化
[[ 1.4443173e-03  8.3994446e-04 -9.4415143e-04 ...  1.8944054e-04
  -4.1765813e-04  1.2399777e-03]
 [ 1.1112873e-04  7.2459027e-04 -7.0364395e-04 ...  6.7487767e-04
  -1.0529784e-03  1.2783859e-03]
 [ 1.4923571e-03  7.1707985e-04 -2.9602324e-04 ... -2.6343833e-04
   6.0850475e-04 -8.7404816e-04]
 ...
 [ 1.0234175e-03 -6.9845311e-04  8.8316134e-05 ...  7.5546751e-04
  -1.4929551e-03 -1.3608179e-03]
 [-1.0456373e-03 -2.6351208e-04 -7.3164358e-04 ...  1.3867704e-03
   3.5324338e-04  1.5639717e-03]
 [ 1.5616795e-03 -5.6039623e-04 -1.1962391e-03 ... -1.4969831e-03
  -9.1569714e-04  8.4922009e-04]] 
 (6752, 300) 次元 最大値 0.0016666665 

sentencepieceで分割した文書をベクトル化
[[ 1.4443173e-03  8.3994446e-04 -9.4415143e-04 ...  1.8944054e-04
  -4.1765813e-04  1.2399777e-03]
 [ 1.1112873e-04  7.2459027e-04 -7.0364395e-04 ...  6.7487767e-04
  -1.0529784e-03  1.2783859e-03]
 [ 1.4923571e-03  7.1707985e-04 -2.9602324e-04 ... -2.6343833e-04
   6.0850475e-04 -8.7404816e-04]
 ...
 [ 1.0234175e-03

## ELMo

双方向LSTMを用いて学習させた言語モデルで、文脈を考慮した単語埋め込み表現が得られます。[実装](https://pypi.org/project/elmoformanylangs/)

（参考）

[大規模日本語ビジネスニュースコーパスを学習したELMo（MeCab利用）モデルの紹介](https://qiita.com/mkt3/items/9577b63900109ff91665)

[大規模日本語ビジネスニュースコーパスを学習したELMo（MeCab利用）モデルの利用方法と精度比較検証](https://qiita.com/kaeru_nantoka/items/bca53a2daea2b29c9b39)

In [ ]:
# 必要なライブラリをインストールします。
%cd '/content/drive/My Drive/'
!pip install overrides
!git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git
!sudo python 'ELMoForManyLangs/setup.py' install

import numpy as np
import pandas as pd
import torch
from ELMoForManyLangs.elmoformanylangs import Embedder
from overrides import overrides
from IPython.display import clear_output
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

/content
  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10174 sha256=7b0156bfbac49e7fb47529042f4654b15398d7f2ea2d44606a0eb53c93f6162e
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
Successfully built overrides
fatal: destination path 'ELMoForManyLangs' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing elmoformanylangs.egg-info/PKG-INFO
writing dependency_links to elmoformanylangs.egg-info/dependency_links.txt
writing requirements to elmoformanylangs.egg-info/requires.txt
writing top-level names to elmoformanylangs.egg-info/top_level.txt
reading manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying elmoformanylangs

（作業）

https://drive.google.com/drive/u/1/folders/1sau1I10rFeAn8BDk8eZDL5qaEjTlNghp

こちらから　単語単位埋め込みモデル　と　文字単位・単語単位埋め込みモデル　とをダウンロードし、マイドライブにアップロードします。

In [ ]:
# ELMo

def serial_mode(txt_batch):
  data = []
  for k in range(len(txt_batch)):
    torch.cuda.empty_cache()
    data.append(char_e.sents2elmo([txt_batch[k]], output_layer=output_layer))
  return data

word_model_path = './単語単位埋め込みモデル'
char_model_path = './文字単位・単語単位埋め込みモデル'

# 文字単位・単語単位埋め込みモデルを読み込み
char_e = Embedder(model_dir=char_model_path, batch_size=64)
output_layer=-1 # モデルの最終段の出力を得る

corpus = np.load('data/mecab_data.npy', allow_pickle=True)

texts = []
new_array = []
n_batch = 25
print('\nコーパス整理中')
for i in range(len(corpus)):
  texts.append(corpus[i].split(' '))

for i in range(len(corpus)//n_batch):
  if i%30 == 0:
    clear_output()
  print('\rELMo変換中\t', 
        str(n_batch*i)+'-'+str(n_batch*(i+1))+'/'+str(len(corpus)), end='\t')
  try:
    data = char_e.sents2elmo(texts[n_batch*i:n_batch*(i+1)], output_layer=output_layer)
    torch.cuda.empty_cache()
    # 可変長ベクトルの平均をとり固定長ベクトルにする
    for j in range(len(data)):
      new_array.append(np.average(data[j], axis=0).reshape(1,-1))
    torch.cuda.empty_cache()
  except:
    # n_batch数の処理でメモリオーバーフローを起こすとき1文書ずつ処理する
    print('serial mode')
    data = serial_mode(texts[n_batch*i:n_batch*i+1])
    for j in range(len(data)):
      new_array.append(np.average(data[j][0], axis=0).reshape(1,-1))
      torch.cuda.empty_cache()
    print('\rELMo変換中\t', 
        str(n_batch*i)+'-'+str(len(corpus))+'/'+str(len(corpus)), end='\t')
data = serial_mode(texts[n_batch*(i+1):])
for j in range(len(data)):
  new_array.append(np.average(data[j][0], axis=0).reshape(1,-1))

new_array = np.vstack(new_array)
mmwrite('data/mecab_ELMo_csr', csr_matrix(new_array))
print('実行結果', new_array.shape, '\n', new_array)

2020-10-19 01:04:44,936 INFO: 1 batches, avg len: 175.2


ELMo変換中	 6000-6025/6752	

2020-10-19 01:04:46,760 INFO: 1 batches, avg len: 213.3


ELMo変換中	 6025-6050/6752	

2020-10-19 01:04:48,625 INFO: 1 batches, avg len: 189.4


...

ELMo変換中	 6700-6725/6752	

2020-10-19 01:05:41,933 INFO: 1 batches, avg len: 179.8


ELMo変換中	 6725-6750/6752	

2020-10-19 01:05:43,697 INFO: 1 batches, avg len: 191.0
2020-10-19 01:05:44,194 INFO: 1 batches, avg len: 55.0


実行結果 (6752, 1024) 
 [[ 0.07609484  0.13494179 -0.1271863  ...  0.35290182  0.30815312
  -0.1912412 ]
 [-0.11880717  0.14558306 -0.10437959 ...  0.4845544   0.20008522
  -0.7493846 ]
 [-0.1683051   0.100697    0.17939211 ...  0.5429394   0.40682924
  -0.01956601]
 ...
 [-0.18878338  0.10764265 -0.23103067 ...  0.4900653  -0.1243621
  -0.45817044]
 [-0.13619576  0.18513697  0.21140532 ...  0.5173607   0.14656053
  -0.21277009]
 [-0.0048144   0.09562767  0.41113737 ...  0.769303    0.00904824
  -0.6708213 ]]


## sentence-transformers日本語版

https://github.com/sonoisa/sentence-transformers

（参考）

[はじめての自然言語処理｜第9回 Sentence BERT による類似文章検索の検証](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part9.html)

In [ ]:
! pip install mecab-python3==0.996.5

     |████████████████████████████████| 17.1MB 202kB/s 


In [ ]:
%cd '/content/drive/My Drive/'
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt

/content
fatal: destination path 'sentence-transformers' already exists and is not an empty directory.
     |████████████████████████████████| 450kB 4.6MB/s 
     |████████████████████████████████| 133kB 12.5MB/s 
     |████████████████████████████████| 890kB 10.3MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 6.7MB 29.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b675deefbdeb3605493636ab48d57ef045e08458fbc9bbdc8a5120d350c7483d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar
%cd '/content/drive/My Drive/sentence-transformers/'
from sentence_transformers import SentenceTransformer
%cd '/content/drive/My Drive/'


--2020-10-19 01:06:24--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2
Resolving www.floydhub.com (www.floydhub.com)... 104.26.1.30, 104.26.0.30, 172.67.72.144, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.26.1.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘sonobe-datasets-sentence-transformers-model.tar’

sonobe-datasets-sen     [                <=> ] 422.28M  22.9MB/s    in 20s     

2020-10-19 01:06:44 (21.5 MB/s) - ‘sonobe-datasets-sentence-transformers-model.tar’ saved [442788352]

./
./training_bert_japanese/
./training_bert_japanese/0_BERTJapanese/
./training_bert_japanese/0_BERTJapanese/added_tokens.json
./training_bert_japanese/0_BERTJapanese/config.json
./training_bert_japanese/0_BERTJapanese/pytorch_model.bin
./training_bert_japanese/0_BERTJapanese/sentence_bert_config.json
./training_bert_japa

2020-10-19 01:06:50,559 INFO: PyTorch version 1.6.0+cu101 available.
2020-10-19 01:06:56,667 INFO: TensorFlow version 2.3.0 available.


/content/drive/My Drive


In [ ]:
%tensorflow_version 2.x
import numpy as np
model_path = '/content/drive/My Drive/training_bert_japanese'
model = SentenceTransformer(model_path, show_progress_bar=False)


2020-10-19 01:07:07,770 INFO: Load pretrained SentenceTransformer: /content/drive/My Drive/training_bert_japanese
2020-10-19 01:07:07,772 INFO: Load SentenceTransformer from folder: /content/drive/My Drive/training_bert_japanese
2020-10-19 01:07:15,260 INFO: loading configuration file /content/drive/My Drive/training_bert_japanese/0_BERTJapanese/config.json
2020-10-19 01:07:15,262 INFO: Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output

In [ ]:
# document vector
import joblib
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

corpus = np.load('data/mecab_data.npy', allow_pickle=True)

vectors = model.encode(corpus)
vectors = np.vstack(vectors)
mmwrite('data/mecab_STtransf_csr', csr_matrix(vectors))
# 再読込、表示
loaddata = mmread('data/mecab_STtransf_csr.mtx').todense()
print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))

[[ 0.30867311  0.72003454 -1.0142187  ... -0.14144777  0.12558933
  -1.9725475 ]
 [ 0.13991801  1.4396574  -0.9894231  ... -0.31704736  0.26261249
  -1.7517378 ]
 [ 0.30623767  0.91992551 -0.36928141 ... -0.45889604 -0.18640393
  -1.1709334 ]
 ...
 [-0.00736655  0.31887358 -0.94143999 ...  0.3203817  -0.06085746
  -1.1184773 ]
 [ 0.90899813 -0.76965106 -0.40118608 ... -0.40824756 -0.54954857
  -0.34403381]
 [ 0.37465972 -0.33253226 -1.408565   ... -0.07329539 -0.50081831
  -1.7956736 ]] 
 (6752, 768) 次元 最大値 4.1483793


## ラベルエンコード、one-hot ベクトル

Tensorflow.Kerasモジュールのテキスト処理クラスである[Tokenizer](tf.keras.preprocessing.text.Tokenizer)に文書を与えることによって、

単語IDの列からなる文書ベクトルが得られます。

このとき、sequenceで各文書の長さをそろえますが、短い文書の後ろに 0 を付ける（zero padding）のが主流です。

長い文書に揃えず、適当な長さで区切って、余った単語を切り捨てる方法もあります。

参考：[Keras Documentation](https://keras.io/ja/preprocessing/text/)

また、[np_utilsクラス](https://keras.io/ja/utils/np_utils/)の to_categorical API に数値nを入力すると、

n番目が1で残りが0のベクトル（one-hotベクトル）が得られます。

参考：[Keras Documentation](https://keras.io/ja/)

In [ ]:
# ラベルエンコード

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  # 文字列をID列に変換
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  list_tokenized = tokenizer.texts_to_sequences(corpus)
  # ID列の長さを揃える
  list_sequence = sequence.pad_sequences(list_tokenized, maxlen=None, dtype='int32', 
                                        padding='post', value=0.0)
  list_tokenized = np.array(list_sequence)
  print(list_tokenized, '\n', list_tokenized.shape, '次元', '最大値', np.max(list_tokenized), '\n')
  # ID列の長さを50で打ち切る
  list_sequence = sequence.pad_sequences(list_tokenized, maxlen=50, dtype='int32', 
                                        padding='post', value=0.0)
  list_tokenized = np.array(list_sequence)
  print(list_tokenized, '\n', list_tokenized.shape, '次元', '最大値', np.max(list_tokenized), '\n')  

MeCabで分割した文書をベクトル化
[[ 59  92   5 ...   0   0   0]
 [155  55   1 ...   0   0   0]
 [ 43  40 202 ...   0   0   0]
 ...
 [  6 133   3 ...   0   0   0]
 [103 132 107 ...   0   0   0]
 [342 343  48 ...   0   0   0]] 
 (50, 307) 次元 最大値 796 

[[ 55   7   1 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [133   3 168 ...   0   0   0]
 [  4 198 559 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 
 (50, 50) 次元 最大値 791 

sentencepieceで分割した文書をベクトル化
[[ 27  49  42 ...   0   0   0]
 [ 27  63  89 ...   0   0   0]
 [ 27  37   3 ...   0   0   0]
 ...
 [ 27   4  42 ...   0   0   0]
 [ 27  92 137 ...   0   0   0]
 [ 27 383 384 ...   0   0   0]] 
 (50, 281) 次元 最大値 1000 

[[  8  49 133 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [ 34  10   4 ...   0   0   0]
 [  1 128   9 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 
 (50, 50) 次元 最大値 992 

3-gramで分割した文書をベクトル化
[[ 256  257  304 ...    0    0    0]
 [2719 2720 2721 ...    0    0

In [ ]:
# one-hotベクトル(1)
# Tensorflowに取り込まれる前のKerasのutilityにあるnp_utilsを用いる方法

from keras.utils import np_utils

index_data = [0, 1, 0, 2, 0, 1, 0, 2]
np_utils.to_categorical(index_data)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
# one-hotベクトル(2)
# 単位行列を生成するNumPyのeye関数で単位行列を生成し、数値をかける方法

import numpy as np

index_data = [0, 1, 0, 2, 0, 1, 0, 2]
print(np.eye(max(index_data)+1)) # 3×3単位行列
np.eye(max(index_data)+1)[index_data]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])